# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tazovsky,67.4667,78.7000,-25.44,90,100,4.73,RU,1738123791
1,1,port-aux-francais,-49.3500,70.2167,5.49,82,66,13.75,TF,1738123792
2,2,east london,-33.0153,27.9116,23.12,85,3,7.96,ZA,1738123795
3,3,roma,43.2128,-75.4557,-6.85,89,100,5.14,US,1738123796
4,4,phayao,19.2500,100.2500,25.83,34,22,1.43,TH,1738123797


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot

map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    size="Humidity",
    color="City",
    tiles="OSM",
    frame_width=700,
    frame_height=500
)
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
ideal_df= city_data_df.loc[(city_data_df["Max Temp"]>=25)&(city_data_df["Humidity"]<50)]

# Drop any rows with null values
ideal_df=ideal_df.dropna()

# Display sample data
ideal_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,phayao,19.2500,100.2500,25.83,34,22,1.43,TH,1738123797
14,14,carnarvon,-24.8667,113.6333,36.13,47,53,0.75,AU,1738123810
153,153,phan rang-thap cham,11.5667,108.9833,27.65,46,34,7.05,VN,1738123992
184,184,ritchie,-29.0380,24.6028,25.73,41,73,4.67,ZA,1738124029
186,186,alice springs,-23.7000,133.8833,38.75,14,0,6.69,AU,1738124032
224,224,mount isa,-20.7333,139.5000,39.87,24,40,5.66,AU,1738124372
252,252,villa regina,-39.1000,-67.0667,29.12,35,66,6.72,AR,1738124410


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df=ideal_df[["City","Country","Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
4,phayao,TH,19.2500,100.2500,34,
14,carnarvon,AU,-24.8667,113.6333,47,
153,phan rang-thap cham,VN,11.5667,108.9833,46,
184,ritchie,ZA,-29.0380,24.6028,41,
186,alice springs,AU,-23.7000,133.8833,14,
224,mount isa,AU,-20.7333,139.5000,24,
252,villa regina,AR,-39.1000,-67.0667,35,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [9]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey":geoapify_key,
    "limit": 20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude=row["Lat"]
    longitude=row["Lng"]
    

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] =  f"circle:{longitude},{latitude},{radius}"
    params["bias"] =  f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url,params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
phayao - nearest hotel: No hotel found
carnarvon - nearest hotel: No hotel found
phan rang-thap cham - nearest hotel: Khách Sạn Ninh Thuận
ritchie - nearest hotel: No hotel found
alice springs - nearest hotel: Aurora Alice Springs
mount isa - nearest hotel: Ibis Styles
villa regina - nearest hotel: Nuevo Cheo


,City,Country,Lat,Lng,Humidity,Hotel Name
4,phayao,TH,19.2500,100.2500,34,No hotel found
14,carnarvon,AU,-24.8667,113.6333,47,No hotel found
153,phan rang-thap cham,VN,11.5667,108.9833,46,Khách Sạn Ninh Thuận
184,ritchie,ZA,-29.0380,24.6028,41,No hotel found
186,alice springs,AU,-23.7000,133.8833,14,Aurora Alice Springs
224,mount isa,AU,-20.7333,139.5000,24,Ibis Styles
252,villa regina,AR,-39.1000,-67.0667,35,Nuevo Cheo


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot
new_map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    size="Humidity",
    color="City",
    tiles="OSM",
    frame_width=700,
    frame_height=500,
)
new_map_plot

# Display the map
# YOUR CODE HERE

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)